# ライブラリインポート

In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import os
import datetime
from sklearn.preprocessing import LabelEncoder
from datetime import timedelta
import seaborn as sns
import collections
import math
import openpyxl
import requests
from bs4 import BeautifulSoup 
import csv

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

%precision 3

'%.3f'

# 気候情報取得地域指定

In [2]:
place_codeA = [14, 21, 23, 31, 32, 33, 35, 34, 36, 54, 56, 55, 48, 41, 57, 42, 43, 40, 52, 51, 49, 45, 53, 50, 46, 68, 69, 61, 60, 67, 66, 63, 65, 64, 73, 72, 74, 71, 81, 82, 85, 83, 84, 86, 88, 87]
place_codeB = [47412, 47423, 47430, 47575, 47582, 47584, 47588, 47590, 47595, 47604, 47605, 47607, 47610, 47615, 47616, 47624, 47626, 47629, 47632, 47636, 47638, 47648, 47651, 47656, 47670, 47741, 47746, 47759, 47761, 47765, 47768, 47770, 47777, 47780, 47887, 47891, 47893, 47895, 47762, 47807, 47813, 47815, 47817, 47819, 47827, 47830]
place_name = ["札幌", "室蘭", "函館", "青森", "秋田", "盛岡", "山形", "仙台", "福島", "新潟", "金沢", "富山", "長野", "宇都宮", "福井", "前橋", "熊谷", "水戸", "岐阜", "名古屋", "甲府", "銚子", "津", "静岡", "横浜", "松江", "鳥取", "京都", "彦根", "広島", "岡山", "神戸", "和歌山", "奈良", "松山", "高松", "高知", "徳島", "下関", "福岡", "佐賀", "大分", "長崎", "熊本", "鹿児島", "宮崎"]   

# アウトプットファイルのパス指定

In [3]:
os.chdir("C://Users/dsuzuki/Desktop/作業用/BPR/スクレイピング/output")

In [4]:
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=1&view=p1"

In [5]:
def str2float(str):
    try:
        return float(str)
    except:
        return 0.0

In [6]:
if __name__ == "__main__":
  #都市を網羅します
  for place in place_name:
    #最終的にデータを集めるリスト (下に書いてある初期値は一行目。つまり、ヘッダー。)
    All_list = [['年月日', '陸の平均気圧(hPa)', '海の平均気圧(hPa)', '降水量(mm)', '平均気温(℃)', '平均湿度(%)', '平均風速(m/s)', '日照時間(h)', '天気概況(昼)', '天気概況(夜)']]
    print(place)
    index = place_name.index(place)
    # for文で2007年~2017年までの11回。
    for year in range(2018,2020):
        #print(year)
        
        for month in range(1,13):
            r = requests.get(base_url%(place_codeA[index], place_codeB[index], year, month))
            r.encoding = r.apparent_encoding

        # 対象である表をスクレイピング。
            soup = BeautifulSoup(r.text)
            rows = soup.findAll('tr',class_='mtx') #タグ指定してclass名を指定するみたい。

        # 表の最初の1~4行目はカラム情報なのでスライスする。(indexだから初めは0だよ)
        # 【追記】2020/3/11 申し訳ございません。間違えてました。
            rows = rows[4:]

        # 1日〜最終日までの１行を網羅し、取得します。
            for row in rows:
                data = row.findAll('td')

          #１行の中には様々なデータがあるので全部取り出す。
          # ★ポイント
                rowData = [] #初期化
                rowData.append(str(year) + "/" + str(month) + "/" + str(data[0].string))
                rowData.append(str2float(data[1].string))
                rowData.append(str2float(data[2].string))
                rowData.append(str2float(data[3].string))
                rowData.append(str2float(data[6].string))
                rowData.append(str2float(data[9].string))
                rowData.append(str2float(data[11].string))
                rowData.append(str2float(data[16].string))
                rowData.append(data[19].string)
                rowData.append(data[20].string)

          #次の行にデータを追加
                All_list.append(rowData)
                
    df_alllist = pd.DataFrame(All_list)
    print(df_alllist.head())
    #都市ごとにデータをファイルを新しく生成して書き出す。(csvファイル形式。名前は都市名)
    with open(place + '.csv', 'w') as file:
        writer = csv.writer(file, lineterminator='\n')
        writer.writerows(All_list)

札幌
          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1       1005.7         1009      9.5      0.4       87        2.9   
2  2018/1/2       1002.7         1006        3      1.1       73        2.2   
3  2018/1/3       1006.3       1009.6      3.5     -0.7       71        2.3   
4  2018/1/4       1011.5       1014.8      4.5     -1.7       75          2   

         7         8           9  
0  日照時間(h)   天気概況(昼)     天気概況(夜)  
1      0.3  雪後みぞれ時々雨  みぞれ時々曇後一時雪  
2      2.5     雪後時々晴       曇後時々雪  
3      0.9      曇時々雪        雪一時曇  
4      0.1      雪時々曇        曇一時雪  
室蘭
          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1       1000.7       1006.9      7.5      4.5       89        6.8   
2  2018/1/2       1000.2       1006.4        4      2.2       78        

          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1          965       1016.4        0        1       89        2.1   
2  2018/1/2        966.1       1017.7        4      0.5       85        2.8   
3  2018/1/3        965.9       1017.8      3.5     -1.2       86          2   
4  2018/1/4        969.2       1021.4        0     -1.3       89        1.5   

         7               8        9  
0  日照時間(h)         天気概況(昼)  天気概況(夜)  
1      0.5  曇時々雪一時雨、あられを伴う     雪時々曇  
2      3.2         雪時々曇一時晴   雪、雷を伴う  
3      2.3               雪        雪  
4      3.3               雪      雪後曇  
宇都宮
          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1        995.1       1012.4        0      3.8       69        2.5   
2  2018/1/2        996.3       1013.7        0        3 

          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1       1019.1       1021.9        0      6.3       53        6.3   
2  2018/1/2       1020.5       1023.3        0      7.1       53        6.3   
3  2018/1/3       1021.4       1024.2      1.5      4.7       62        4.7   
4  2018/1/4       1020.9       1023.7        4        3       92        1.6   

         7                 8                 9  
0  日照時間(h)           天気概況(昼)           天気概況(夜)  
1      1.3       曇一時雨、あられを伴う           曇時々晴一時雨  
2      6.3              晴一時曇       曇時々雨、あられを伴う  
3      1.3  曇時々雨一時みぞれ、あられを伴う  曇時々みぞれ一時雪、あられを伴う  
4      1.3  雨時々曇一時みぞれ、あられを伴う              曇一時雨  
鳥取
          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1       1019.1       1021.1      7.5      5.3       70      

          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1       1020.4       1021.2        0      6.9       47        4.2   
2  2018/1/2         1022       1022.8        0      7.6       45        3.8   
3  2018/1/3       1021.2         1022        0      6.4       50        4.6   
4  2018/1/4       1022.1       1022.9        0        5       55        3.2   

         7        8        9  
0  日照時間(h)  天気概況(昼)  天気概況(夜)  
1      8.3        晴        晴  
2        9        晴        晴  
3        2     晴時々曇     晴一時曇  
4      8.3        晴     曇一時晴  
下関
          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1       1021.8       1024.2        0      7.1       48          5   
2  2018/1/2       1023.4       1025.8        0      8.3       51        4.1   
3  2018/1/3       10